In [1]:
import os
import sys
import pandas as pd

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(os.path.join(current_dir), '..', 'main')))

from datasets import dataloader, process
from feature_engineering.fe import data_preprocessing
from algorithm import MDRSmapClassifier

I. Load data

In [2]:
##############################
TICKER = 2330
TP = 10
START_PRED_DATE = '2023-07-01'
BACKTRADE_END_DATE = '2023-10-31'
##############################

In [3]:
ticker_data = dataloader.get_ticker_data(ticker=TICKER, after_date='2020-11-17')
ticker_data = process.processing(ticker_data)

II. Feature enineering

In [4]:
_, Library, Prediction, actual_start_pred_date = data_preprocessing(ticker_data=ticker_data, TP=TP, start_pred_date=START_PRED_DATE)

In [5]:
Pred_1 = Prediction.iloc[0,:].to_frame().T
Pred_1['Date'] = pd.to_datetime(Pred_1['Date'])
Pred_1['Date'] = Pred_1['Date'].dt.strftime('%Y-%m-%d')
Pred_1

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_10
593,2023-07-03,578.0,580.0,576.0,579.0,15118,97.0,2353.0,-305.0,1401,582,2.5477,576.701182,575.940219,566.756798,9.183421,11.833896,33.311646,0.0


In [6]:
Pred_2 = Prediction.iloc[1,:].to_frame().T
Pred_2['Date'] = pd.to_datetime(Pred_2['Date'])
Pred_2['Date'] = Pred_2['Date'].dt.strftime('%Y-%m-%d')
Pred_2

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_10
594,2023-07-04,585.0,585.0,580.0,585.0,17777,84.0,4805.0,-1348.0,-13,1767,2.7068,578.360946,577.334031,568.108146,9.225886,11.312294,30.627646,0.0


In [7]:
Pred_3 = Prediction.iloc[2,:].to_frame().T
Pred_3['Date'] = pd.to_datetime(Pred_3['Date'])
Pred_3['Date'] = Pred_3['Date'].dt.strftime('%Y-%m-%d')
Pred_3

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_10
595,2023-07-05,589.0,589.0,579.0,582.0,15554,-50.0,-890.0,-503.0,-1092,-790,2.6473,579.088757,578.051873,569.137172,8.914701,10.832775,17.707207,0.0


III. Realtime prediction

In [8]:
##############################
TARGET = f'y_{TP}'
KN = 100
EMAX = 10
MAXLAG = 10
THETA_SEQ = [1,2,4,7,11,16,22]
VALID_INTERVAL = 60
##############################

In [9]:
parameter = {'Library': Library,
             'Prediction': Pred_1,
             'target': TARGET, 
             'valid_interval': VALID_INTERVAL, 
             'tp': TP, 
             'ticker': TICKER, 
             'E_max': EMAX, 
             'max_lag': MAXLAG, 
             'kn': KN, 
             'theta_seq': THETA_SEQ}

y_pred, param_dir, EDM_result = MDRSmapClassifier.realtime_train(**parameter)

optimal embedding dimenson has been found: 1

now applying CCM feature selection...
now applying Muti-view random search...
now applying S-map...
now applying SAGA logistic(elastic-net)...

prediction: 0.0
Parameters saved to /Users/yitsung/Desktop/MDRSmapClassifier/main/algorithm/../models/2330_10_param.pkl


In [10]:
pred_parameter = {'Prediction': Pred_2,
                  'file_dir': param_dir,
                  'Library': Library,
                  'EDM_result': EDM_result,
                  'target': TARGET, 
                  'tp': TP, 
                  'max_lag': MAXLAG}

y_pred = MDRSmapClassifier.realtime_pred(**pred_parameter)

Parameter loaded from /Users/yitsung/Desktop/MDRSmapClassifier/main/algorithm/../models/2330_10_param.pkl
prediction: 0.0
